---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [4]:
import wandb

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utilities as tfu

In [6]:
!nvidia-smi

Tue May  9 22:17:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:68:00.0 Off |                  N/A |
|  0%   33C    P8    18W / 250W |      1MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
tf.config.list_physical_devices()

2023-05-09 22:17:17.469513: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

IndexError: list index out of range

---
# Load Data

In [7]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/dnabert-pretrain/dnabert-pretrain-64dim:latest").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model.load_weights(model_path + "/model.h5")
pretrained_model

wandb:   5 of 5 files downloaded.  


In [8]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples-complete:latest").download()
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples-complete:latest, 4079.09MB. 420 files... 
wandb:   420 of 420 files downloaded.  
Done. 0:0:12.4


'./artifacts/dnasamples-complete:v0/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [11]:
split_ratios = [0.8, 0.2]
set_len = 1000
sequence_len = 150
kmer = 3
batch_size = [1,1]
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [12]:
rng.shuffle(random_samples)

In [13]:
trimmed_samples, (train_dataset, val_dataset) = DnaSampleGenerator.split(samples=random_samples[0:5], split_ratios=split_ratios, subsample_length=set_len, sequence_length=sequence_len, kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

--- 
# Batch Parameters

In [14]:
block_size = 250
max_set_len = set_len
max_files = len(train_dataset.samples)
max_files

5

In [15]:
if block_size-2 > set_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [16]:
print(max_set_len)

1000


---
# Create Embeddings

In [17]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [18]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)
        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        result = tf.reshape(encoded, (batch_size, subsample_size, -1))
        return result
    
    def call(self, data):
        return  self.modify_data_for_input(data)

# Cache Memory

In [19]:
def Cache_Memory(current_state, previous_state, memory_length):
    if memory_length is None or memory_length == 0:
        return None, None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
            excess = current_state[:, :-memory_length, :]
        else:
            concatanted =  tf.concat([previous_state, current_state], 1)
            new_mem = concatanted[:, -memory_length:, :]
            excess = concatanted[:,:-memory_length,:]
            
    return new_mem, excess

---
# Attention

In [20]:
class Attention(keras.layers.Layer):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)

    def call(self, data, mems=None):
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [21]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_compressed_seeds,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_compressed_seeds = num_compressed_seeds
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.compress = Set_Transformer.PoolingByMultiHeadAttention(
            num_seeds=self.num_compressed_seeds,
            embed_dim=self.embed_dim,
            num_heads=self.num_heads,
            use_layernorm=self.use_layernorm,
            pre_layernorm=self.pre_layernorm, 
            use_keras_mha=self.use_keras_mha, 
            is_final_block=True)
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)
    
    def call(self,
             content_stream,
             state=None,
             compressed=None):
        
        memories = tf.concat((state, compressed), axis=1)
        
        attention_output = self.attention_layer(content_stream, memories)

        return attention_output       

---
# Transformer XL

In [22]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_compressed_seeds,
                 compressed_len,
                 mem_len,
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_compressed_seeds = num_compressed_seeds
        self.compressed_len = compressed_len
        self.mem_len = mem_len
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_compressed_seeds,
                                        self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None,
             compressed=None):
        
        new_mems = []
        new_compressed = []

        if state is None:
            state = [None] * self.num_layers
            
        if new_compressed is None:
            new_compressed = [None] * self.num_layers
            
        for i, transformer_xl_layer in enumerate(self.transformer_xl_layers):
            if self.mem_switched == False:
                mems_append, mems_excess = Cache_Memory(content_stream, state[i], self.mem_len)
                new_mems.append(tf.stop_gradient(mems_append))
                
                #Perform attention between current segment and uncompressed trimmed memory
                uncompressed_attention = transformer_xl_layer.attention_layer(tf.stop_gradient(content_stream), tf.stop_gradient(mems_excess))
                
                #Compress the extra memories
                compressed_mems = transformer_xl_layer.compress(tf.stop_gradient(mems_excess))
                
                compressed_append, _ = Cache_Memory(compressed_mems, compressed[i], self.compressed_len)
                new_compressed.append(compressed_append)
            
                #Perform attention between current segment and compressed trimmed memory
                compressed_attention = transformer_xl_layer.attention_layer(tf.stop_gradient(content_stream), compressed_mems)
                
                #Calculate attention reconstruction loss
                loss = tf.linalg.norm(uncompressed_attention-compressed_attention)
                
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i], compressed=compressed[i])
            
            content_stream = self.output_dropout(transformer_xl_output)

        output_stream = content_stream
        return output_stream, new_mems, new_compressed, loss

---
# Xl Model Class

In [54]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, num_compressed_seeds, compressed_len, mem_len, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, dropout_rate, num_seeds_output, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.num_compressed_seeds = num_compressed_seeds
        self.compressed_len = compressed_len
        self.mem_len = mem_len
        self.max_files = max_files
        self.encoder = encoder
        self.block_size = block_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.num_seeds_output = num_seeds_output
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.embedding_layer = Create_Embeddings(self.encoder)

        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_compressed_seeds,
                                            self.compressed_len,
                                            self.mem_len,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds_output,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((self.embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
    
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred, loss_compressed = self(x, return_loss=True, training=True) 
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            tf.print("LossC", loss_compressed, "Loss", loss)
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(loss+loss_compressed, trainable_vars)
            grad_check = tf.debugging.check_numerics(gradients[0], 'check_numerics caught bad gradients')
            with tf.control_dependencies([grad_check]):
                self.optimizer.apply_gradients(zip(gradients, trainable_vars))
                self.compiled_metrics.update_state(y, y_pred)

            return {m.name: m.result() for m in self.metrics}
        
    def test_step(self, data):
        x, y = data

        y_pred, loss_compressed = self(x, return_loss=True, training=False)

        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def call(self, x, return_loss=False, training=None):  
        
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
        compressed = tf.zeros((self.num_layers, tf.shape(x)[0], self.compressed_len, self.embed_dim))

        embeddings = self.embedding_layer(x)

        linear_transform = self.linear_layer(embeddings)

        losses = 0
        
        for i in range(0, self.max_set_len, self.block_size):
            block = linear_transform[:,i:i+self.block_size]
            
            output, mems, compressed, loss = self.transformer_xl(content_stream=block, state=mems, compressed=compressed)
            losses = losses + loss
            
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        if return_loss:
            return output, losses
        
        return output

---
# Xl Parameters

In [55]:
#Xl Parameters
encoder = pretrained_encoder
mem_switched = False
num_seeds_compressed = 200
compressed_len = 250
mem_len = 250
num_induce = 0
embed_dim = 64
num_layers = 4
num_heads = 4
dropout_rate = 0.01
num_seeds_output = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

---
# Create Models

In [56]:
model = XlModel(mem_switched, num_seeds_compressed, compressed_len, mem_len, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, dropout_rate, num_seeds_output, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Adam(1e-3), metrics = keras.metrics.SparseCategoricalAccuracy())

In [57]:
model(train_dataset[0][0])

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.02070339, 0.24968238, 0.09813999, 0.04604487, 0.5854294 ]],
      dtype=float32)>

In [58]:
epochs=4

In [59]:
%load_ext tensorboard
from datetime import datetime

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [60]:
history = model.fit(x=train_dataset, validation_data=val_dataset, epochs=epochs, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/4
LossC 43.0081329 Loss 2.8927021
12/20 [=================>............] - ETA: 1:34 - loss: 1.8906 - sparse_categorical_accuracy: 0.5000LossC 7.49088287 Loss 1.16497552


KeyboardInterrupt: 

In [ ]:
!tensorboard dev upload \
  --logdir logs \
  --name "Sample op-level graph" \
  --one_shot

In [45]:
import pickle
filename = "Test.h5"

In [74]:
with open(filename, "wb") as f:
    pickle.dump(model.get_weights(), f)

In [78]:
with open(filename, "rb") as f:
    model.set_weights(pickle.load(f))